In [6]:
import json, numpy as np, pretty_midi as pm
from IPython.display import Audio, display

VOCAB_PATH = "../data/processed/dataset_2/vocab_topN.json"
SAMPLES_JSONL = "../outputs/samples_bars.jsonl"
SF2_PATH = r"C:\Soundfonts\FluidR3_GM.sf2"  # <-- ajusta a tu ruta

from pathlib import Path
import json

# Muestra desde dónde está corriendo el notebook
CWD = Path().resolve()
print("CWD:", CWD)

def find_first(paths):
    """Devuelve el primer Path que exista según el CWD actual."""
    for p in paths:
        pp = (CWD / p).resolve()
        print("probando:", pp, "->", pp.exists())
        if pp.exists():
            return pp
    raise FileNotFoundError("Ninguna de las rutas existe:\n" + "\n".join(map(str, paths)))

# Rutas posibles (si el CWD es el raíz del repo o la carpeta notebooks/)
VOCAB_PATH = find_first([
    "data/processed/dataset_2/vocab_topN.json",      # si CWD es raíz del repo
    "../data/processed/dataset_2/vocab_topN.json",   # si CWD es notebooks/
])

SAMPLES_JSONL = find_first([
    "outputs/samples_bars.jsonl",
    "../outputs/samples_bars.jsonl",
])

print("VOCAB_PATH =", VOCAB_PATH)
print("SAMPLES_JSONL =", SAMPLES_JSONL)



CWD: C:\Users\Djcho\Documents\Repositories\Polyrhythmia\model_lstm
probando: C:\Users\Djcho\Documents\Repositories\Polyrhythmia\model_lstm\data\processed\dataset_2\vocab_topN.json -> True
probando: C:\Users\Djcho\Documents\Repositories\Polyrhythmia\model_lstm\outputs\samples_bars.jsonl -> True
VOCAB_PATH = C:\Users\Djcho\Documents\Repositories\Polyrhythmia\model_lstm\data\processed\dataset_2\vocab_topN.json
SAMPLES_JSONL = C:\Users\Djcho\Documents\Repositories\Polyrhythmia\model_lstm\outputs\samples_bars.jsonl


In [7]:
def load_vocab(vocab_path: Path):
    with open(vocab_path, "r", encoding="utf-8") as f:
        voc = json.load(f)

    # Soporta dos formatos:
    # A) {"000000000":0, "100000000":1, ...}
    # B) {"bitmask2id": {...}, "id2bitmask": {...}, "unk_id": 127}
    if isinstance(voc, dict) and "bitmask2id" in voc:
        bitmask2id = {k: int(v) for k, v in voc["bitmask2id"].items()}
        id2bitmask = {int(k): v for k, v in voc["id2bitmask"].items()}
        unk_id = int(voc.get("unk_id")) if voc.get("unk_id") is not None else None
    else:
        bitmask2id = {k: int(v) for k, v in voc.items()}
        id2bitmask = {v: k for k, v in bitmask2id.items()}
        unk_id = bitmask2id.get("UNK")

    return id2bitmask, unk_id

In [10]:
import json
from pathlib import Path
from src.utils.export_to_midi import load_vocab, read_jsonl_bars, export_bars_to_midi
import pretty_midi
import IPython.display as ipd

# --- Rutas ---
VOCAB_PATH = "data/processed/dataset_2/vocab_topN.json"
SAMPLES_JSONL = "outputs/samples_bars.jsonl"
TEMP_MIDI = "outputs/temp_preview.mid"

# --- Cargar vocabulario y muestras ---
bitmask2id, id2bitmask, unk_id = load_vocab(VOCAB_PATH)
bars = read_jsonl_bars(SAMPLES_JSONL)
print(f"Número de loops generados: {len(bars)}")

# --- Tomar 8 compases y exportar ---
export_bars_to_midi(bars[:8], id2bitmask, TEMP_MIDI, bpm=100)

# --- Reproducir MIDI ---
midi_data = pretty_midi.PrettyMIDI(TEMP_MIDI)
audio_data = midi_data.fluidsynth(
    sf2_path="C:/tools/fluidsynth/soundfonts/FluidR3_GM.sf2"
)
ipd.Audio(audio_data, rate=44100)


Número de loops generados: 32
[OK] MIDI escrito en: outputs/temp_preview.mid


ImportError: fluidsynth() was called but pyfluidsynth is not installed.

In [8]:
CLASS_PITCHES = [36, 38, 42, 46, 41, 45, 48, 49, 51]  # KICK,SNARE,HHc,HHo,TOML,TOMM,TOMH,CRASH,RIDE

def bars_to_pretty_midi(bars_ids, id2bitmask, bpm=100, note_len=0.06, velocity=95):
    midi = pm.PrettyMIDI(initial_tempo=bpm)
    drum = pm.Instrument(program=0, is_drum=True, name="Drums")

    spb = 60.0 / float(bpm)
    bar_dur = 4.0 * spb
    step = bar_dur / 16.0

    t0 = 0.0
    for bar in bars_ids:
        # ids -> bitmask strings
        bitmasks = []
        for tid in bar:
            bm = id2bitmask.get(tid, "000000000")
            bm = bm if isinstance(bm, str) else format(int(bm), "09b")
            bm = bm.zfill(9)
            bitmasks.append(bm)

        for i, bm in enumerate(bitmasks):
            for cls_idx, ch in enumerate(bm):
                if ch == "1":
                    pitch = CLASS_PITCHES[cls_idx]
                    start = t0 + i * step
                    end = start + note_len
                    drum.notes.append(pm.Note(velocity=velocity, pitch=pitch, start=start, end=end))
        t0 += bar_dur

    midi.instruments.append(drum)
    return midi

midi_obj = bars_to_pretty_midi(bars[:8], id2bitmask, bpm=100)  # reproduce 8 compases


NameError: name 'bars' is not defined

In [ ]:
# Requiere pyfluidsynth instalado y SF2_PATH válido
audio = midi_obj.fluidsynth(sf2_path=SF2_PATH, sample_rate=44100)
display(Audio(audio, rate=44100))


In [ ]:
midi_obj.write("../outputs/notebook_preview.mid")
print("MIDI guardado en outputs/notebook_preview.mid")


In [ ]:
def read_jsonl_bars(jsonl_path: Path):
    bars = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            bars.append(obj["tokens"])  # lista de 16 ids
    return bars

id2bitmask, unk_id = load_vocab(VOCAB_PATH)
bars = read_jsonl_bars(SAMPLES_JSONL)
print("Ejemplos:", len(bars), " | primer compás (len):", len(bars[0]))
